<a href="https://colab.research.google.com/github/flipfelly/Banco_de_dados_AWS_RDS/blob/main/Banco_de_dados_AWS_RDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criando um Banco de Dados Postgres AWS RDS e Populando

Para a execução deste código, é necessário que você já tenha uma conta na AWS, um bucket S3, e um banco de dados RDS Postgres criado com acesso público permitido.

---

## Como criar um banco de dados RDS

1. No ambiente AWS, pesquise por **RDS**.
2. Clique em **Criar banco de dados**.
3. Selecione a opção de criação **Padrão**.
4. Em **Opções de mecanismo**, escolha **Postgres** e a versão mais adequada à sua necessidade.
5. Em **Modelos**, escolha a opção **Free Tier** (exercício para fins de aprendizado).
6. Escolha o modelo **Autogerenciado** e defina uma senha (a qual será referida como **SENHA ESCOLHIDA**).
7. **Atenção:** Marque **Sim** na opção **Acesso público**.
8. Desmarque **Insights de Performance** para evitar gastos.
9. Clique em **Criar banco de dados**.

⚠️ **Observação**: Não é necessário alterar as demais configurações, mas revise todas para garantir que você está utilizando o serviço dentro do plano gratuito.

---

## Como criar um bucket S3

1. No ambiente AWS, pesquise por **S3**.
2. Clique em **Criar bucket**.
3. Dê um nome único globalmente ao seu bucket.
4. Desmarque a opção **Bloquear todo o acesso público** (necessário para este exercício).
5. Não altere mais nenhuma configuração, a menos que seja necessário para seu caso.
6. Clique em **Criar bucket**.

### Como colocar arquivos no bucket

1. Clique no nome do bucket (presumindo que você já está na página do S3).
2. Clique em **Criar uma pasta** e dê um nome a ela (neste exercício, a pasta se chama **imagens**).
3. Acesse a pasta criada.
4. Clique em **Carregar**.
5. Clique em **Adicionar arquivos** e selecione os arquivos desejados (as imagens podem estar em uma pasta local).
6. Clique em **Carregar**.


In [ ]:
import psycopg2 #essa biblioteca ajuda a interagir com o postgres atraves do python

### Explicações sobre a célula:

1. **`psycopg2.connect()`**  
   Este método da biblioteca `psycopg2` estabelece uma conexão com o banco de dados PostgreSQL. Ele requer os seguintes parâmetros:  
   - **`host`**: O endereço do banco de dados. Aqui, você deve inserir o **endpoint** do banco de dados RDS.

---

        ### Como encontrar o endpoint do banco de dados:
          1. Acesse a página do **RDS** no console AWS.  
          2. Clique em **Banco de Dados**.  
          3. Selecione o seu banco de dados na lista.  
          4. Vá até a seção **Segurança e Conexão** e copie o campo **Endpoint**.  

        **Pronto, simples, não?**

---

2. **`database`**  
   No momento, use o banco de dados padrão **`postgres`**, pois ainda não criamos o banco de dados desejado. Nesta célula, criaremos um novo banco chamado **`inventario`**.  

3. **`user`**  
   O usuário configurado ao criar o banco no RDS. Por padrão, ao selecionar o mecanismo PostgreSQL, o usuário é **`postgres`**.

4. **`password`**  
   A senha criada durante a configuração do banco no modo autogerenciado. Utilize essa senha para a conexão.

---

### Resolvendo problemas de conexão

Caso ocorra algum problema de conexão e você tenha marcado "Sim" para acesso público, siga os passos abaixo:

1. Clique no seu banco de dados (presumindo que você já está na página do banco de dados no RDS).
2. Em **Segurança**, clique no **Grupo de segurança da VPC**.
3. Selecione o grupo de segurança associado.
4. Acesse **Regras de entrada** → **Editar regras de entrada**.
5. Clique em **Adicionar regra**.
6. Em **Tipo**, selecione **Todo tráfego**.
7. Em **Origem**, selecione **Qualquer local (IPv4)**.
8. Salve as regras.

In [ ]:
con = psycopg2.connect(host ='database-1.cdk6a8eq4oc8.us-east-1.rds.amazonaws.com', database = 'postgres', user = 'postgres', password = 'sua senha')

con.autocommit = True #Garante que as alterações sejam aplicadas imediatamente, sem a necessidade de um `commit` explícito.

cur = con.cursor()#Cursor para realizar as consultas

cur.execute('create database inventario;') #Envia o comando SQL para criar o banco de dados chamado `inventario`.

con.close()


###Explicação sobre a célula:
- Aqui estamos nos conectando com o banco de dados "Inventario", criado na célula anterior.
- Criamos tambem uma tabela chamada "arquivos"

In [ ]:
con = psycopg2.connect(host ='database-1.cdk6a8eq4oc8.us-east-1.rds.amazonaws.com', database = 'inventario', user = 'postgres', password = 'sua senha')

con.autocommit = True

cur = con.cursor()

cur.execute('create table arquivos (idarquivo INT, nomearquivo VARCHAR(256));')

con.close() #Fecha a conexão com o banco de dados, liberando recursos.

In [ ]:
!pip install boto3 #Instalando a biblioteca boto3, que vai nos auxiliar na conexão com os serviços da AWS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.4 MB/s eta 0:00:00


### Conectando com o Bucket S3

Aqui vamos apenas conhecer quais arquivos temos no bucket

1. **Configurações iniciais**:
   - **`service_name`**: Define o nome do serviço AWS com o qual vamos estabelecer conexão, neste caso, **S3**.  
   - **`region_name`**: A região onde o bucket foi criado. No meu exemplo, é a **América do Sul - São Paulo**.  
   - **`aws_access_key_id` e `aws_secret_access_key`**: Suas credenciais de acesso da AWS.  
   - **`bucket`**: Recebe o nome do bucket criado no S3.  
   - **`prefix`**: Define a pasta criada dentro do bucket.

---

### Como encontrar a `region_name`:
1. Acesse a página do **S3** no console AWS.  
2. Clique no **bucket** que deseja utilizar.  
3. Vá até a aba **Propriedades**.  
4. Na seção **Visão Geral do Bucket**, localize **Região AWS**.  
5. Copie o código da região.

**Pronto! Agora você tem a `region_name` configurada.**

---

### Como obter suas `credenciais AWS`:
1. No canto superior direito da tela, encontre o **nome do usuário**.  
2. Clique na **setinha** ao lado do nome.  
3. Selecione **Credenciais de Segurança**.  
4. Localize a seção **Chaves de Acesso**.  
5. Crie uma nova chave de acesso.  
6. Salve o arquivo com a chave ou copie os detalhes.

⚠️ **Atenção**: Você só poderá visualizar a chave de acesso no momento da criação. Guarde-a em local seguro!


In [ ]:
import boto3
import psycopg2
import io

s3 = boto3.resource(service_name = 's3', region_name = 'sa-east-1', aws_access_key_id = 'seu ID de acesso', aws_secret_access_key = 'sua chave de acesso')

bucket = 'img-engdados-python'
prefix = 'imagens/'

for objects_s3 in s3.Bucket(bucket).objects.filter(Prefix = prefix): #percorre os objetos do bucket e nos retorna os nomes
  if objects_s3.key.endswith('jpg') or objects_s3.key.endswith('JPG'):
    filename = objects_s3.key.split('/')[1]
    print(filename)

avenue-g9ad8b9b60_640.jpg
dandelion-ge4c90edd8_640.jpg
fantasy-g95f970a56_640.jpg
garden-g42e486784_640.jpg
pink-ge82d54651_640.jpg
road-g37132565b_640.jpg
road-g41ea28d46_640.jpg
sunset-gac16749a1_640.jpg
tree-g386d6021c_640.jpg
tree-gd34ff2fcb_640.jpg


### Conectando S3 com PostgreSQL e Inserindo Dados no Banco

Aqui utilizamos os codigos das celulas ja executadas, em conjunto, para inserir os dados no banco

### Iteração pelos Arquivos do S3:
1. Através do **boto3**, acessamos os arquivos do bucket filtrando pelo prefixo definido.  
2. Verificamos se os arquivos possuem extensão **.jpg** ou **.JPG**.  
3. Extraímos o nome do arquivo usando o método **`split`**.  
4. Cada arquivo recebe um identificador único **`id`**, que é incrementado no loop.  
5. Executamos um comando **SQL INSERT** no banco para armazenar o ID e o nome do arquivo na tabela **`arquivos`**.  

---

### Observação:
Certifique-se de que:  
- A tabela **`arquivos`** foi criada previamente no banco de dados com os campos **`idarquivo`** (INT) e **`nomearquivo`** (VARCHAR).  
- Suas credenciais AWS e PostgreSQL estejam corretas e seguras.  


In [ ]:
import boto3
import psycopg2
import io

s3 = boto3.resource(service_name = 's3', region_name = 'sa-east-1', aws_access_key_id = ' seu id de acesso ', aws_secret_access_key = 'sua chave de acesso')

bucket = 'img-engdados-python'
prefix = 'imagens/'

con = psycopg2.connect(host ='database-1.cdk6a8eq4oc8.us-east-1.rds.amazonaws.com', database = 'inventario', user = 'postgres', password = 'sua senha')

con.autocommit = True

cur = con.cursor()

id = 0

for objects_s3 in s3.Bucket(bucket).objects.filter(Prefix = prefix):
  if objects_s3.key.endswith('jpg') or objects_s3.key.endswith('JPG'):
    filename = objects_s3.key.split('/')[1]
    id += 1
    cur.execute("insert into arquivos (idarquivo, nomearquivo) values ("+str(id)+", '"+filename+"')")

con.close()

### Certificando que a tabela foi preenchida

In [ ]:
con = psycopg2.connect(host ='database-1.cdk6a8eq4oc8.us-east-1.rds.amazonaws.com', database = 'inventario', user = 'postgres', password = 'sua senha')

con.autocommit = True

cur = con.cursor()

cur.execute('select * from arquivos;')

recset = cur.fetchall()

for rec in recset:
  print(rec)

con.close()

(1, 'avenue-g9ad8b9b60_640.jpg')
(2, 'dandelion-ge4c90edd8_640.jpg')
(3, 'fantasy-g95f970a56_640.jpg')
(4, 'garden-g42e486784_640.jpg')
(5, 'pink-ge82d54651_640.jpg')
(6, 'road-g37132565b_640.jpg')
(7, 'road-g41ea28d46_640.jpg')
(8, 'sunset-gac16749a1_640.jpg')
(9, 'tree-g386d6021c_640.jpg')
(10, 'tree-gd34ff2fcb_640.jpg')
